In [1]:
import pandas as pd
import os
import sys

In [2]:
dbs_path = "/home/ubuntu/poppunk/poppunk_db"
prev_db = "GPS_v9"
new_db = "GPS_v9_1"

In [3]:
prev_db_poppunk_cluster_path = os.path.join(dbs_path, prev_db, f"{prev_db}_clusters.csv")
prev_db_gpsc_path = os.path.join(dbs_path, f"{prev_db}_external_clusters.csv")

new_db_poppunk_cluster_path = os.path.join(dbs_path, new_db, f"{new_db}_clusters.csv")
new_db_gpsc_path = os.path.join(dbs_path, new_db, f"{new_db}_external_clusters.csv")

In [4]:
df_prev_db_poppunk_cluster = pd.read_csv(prev_db_poppunk_cluster_path, dtype=str, keep_default_na=False)
df_prev_db_gpsc = pd.read_csv(prev_db_gpsc_path, dtype=str, keep_default_na=False)

df_new_db_poppunk_cluster = pd.read_csv(new_db_poppunk_cluster_path, dtype=str, keep_default_na=False)
df_new_db_gpsc = pd.read_csv(new_db_gpsc_path, dtype=str, keep_default_na=False)

In [5]:
prev_gpscs_history = set(df_prev_db_gpsc["merge_history"])
# Drop 235;9 as it is an exceptional case
prev_gpscs_history.remove("235;9")
prev_gpscs_history_splitted = {int(gpsc_splitted) for gpsc_merged in prev_gpscs_history for gpsc_splitted in gpsc_merged.split(";")}

prev_poppunk_clusters = set(df_prev_db_poppunk_cluster["Cluster"])
prev_poppunk_clusters_splitted = {int(poppunk_cluster_splitted) for poppunk_cluster_merged in prev_poppunk_clusters for poppunk_cluster_splitted in poppunk_cluster_merged.split("_")}

new_poppunk_clusters = set(df_new_db_poppunk_cluster["Cluster"])
new_poppunk_clusters_splitted = {int(poppunk_cluster_splitted) for poppunk_cluster_merged in new_poppunk_clusters for poppunk_cluster_splitted in poppunk_cluster_merged.split("_")}

In [6]:
prev_poppunk_clusters_min = min(prev_poppunk_clusters_splitted)
prev_poppunk_clusters_max = max(prev_poppunk_clusters_splitted)

print(f"PopPUNK Cluster range used in previous database {prev_db}: {prev_poppunk_clusters_min} - {prev_poppunk_clusters_max}")

prev_gpsc_history_min = min(prev_gpscs_history_splitted)
prev_gpsc_history_max = max(prev_gpscs_history_splitted)

print(f"GPSC range used in previous database {prev_db}: {prev_gpsc_history_min} - {prev_gpsc_history_max}\n")

diff_poppunk_clusters_and_gpsc_max = prev_poppunk_clusters_max - prev_gpsc_history_max
print(f"The difference between maximum PopPUNK Cluster and GPSC range in {prev_db}: {diff_poppunk_clusters_and_gpsc_max}\n")

PopPUNK Cluster range used in previous database GPS_v9: 1 - 1231
GPSC range used in previous database GPS_v9: 1 - 1122

The difference between maximum PopPUNK Cluster and GPSC range in GPS_v9: 109



In [7]:
new_poppunk_new_clusters = new_poppunk_clusters_splitted - prev_poppunk_clusters_splitted

new_poppunk_new_clusters_min = min(new_poppunk_new_clusters)
new_poppunk_new_clusters_max = max(new_poppunk_new_clusters)

print(f"Number of samples with unassigned GPSC in new database {new_db}: {len(df_new_db_gpsc[df_new_db_gpsc["GPSC"] == "NA"])}")
print(f"New PopPUNK Cluster range used in new database {new_db}: {new_poppunk_new_clusters_min} - {new_poppunk_new_clusters_max} (Length: {new_poppunk_new_clusters_max - new_poppunk_new_clusters_min + 1})")

Number of samples with unassigned GPSC in new database GPS_v9_1: 4
New PopPUNK Cluster range used in new database GPS_v9_1: 1232 - 1234 (Length: 3)


In [8]:
df_new_db_gpsc = df_new_db_gpsc.merge(df_new_db_poppunk_cluster.rename(columns={"Taxon": "sample"}), on="sample", how="left")

gpsc_unassigned_mask = df_new_db_gpsc['GPSC'] == "NA"
df_new_db_gpsc.loc[gpsc_unassigned_mask, "GPSC"] = df_new_db_gpsc.loc[gpsc_unassigned_mask, "Cluster"].astype(int) - diff_poppunk_clusters_and_gpsc_max


In [9]:
df_new_db_gpsc = df_new_db_gpsc.astype(str)

In [10]:
df_new_db_gpsc = df_new_db_gpsc[["sample", "GPSC"]].merge(df_prev_db_gpsc[["sample", "merge_history"]], on="sample", how="left")
df_new_db_gpsc["merge_history"] = df_new_db_gpsc["merge_history"].fillna(df_new_db_gpsc["GPSC"])

In [11]:
merged_gpscs = set()

for index, row in df_new_db_gpsc.iterrows():
    # Skip 235;9 in merge cluster joint to keep it isolated
    if row["GPSC"] == "235;9":
        continue
    gpsc_splitted = set(row["GPSC"].split(';'))
    merge_history_splitted = set(row["merge_history"].split(';'))

    base_gpsc = min(gpsc_splitted, key=int)
    if base_gpsc not in merge_history_splitted:
        sys.exit(f"Error:{base_gpsc} not found in merge_history {merge_history_splitted} of {row["sample"]}")

    merged_gpscs.add(";".join(sorted(gpsc_splitted.union(merge_history_splitted), key=int)))

In [12]:
merged_gpscs_with_merge_clusters = [set(gpsc.split(';')) for gpsc in merged_gpscs if ';' in gpsc]

In [13]:
new_merge = True

while new_merge:
    new_merge = False
    next_list = []
    while merged_gpscs_with_merge_clusters:
        cur_merged_gpsc = merged_gpscs_with_merge_clusters.pop()
        for i, merged_gpsc in enumerate(next_list):
            if cur_merged_gpsc.intersection(merged_gpsc):
                next_list[i] = cur_merged_gpsc.union(merged_gpsc)
                new_merge = True
                break
        else:
            next_list.append(cur_merged_gpsc)
    merged_gpscs_with_merge_clusters = next_list

In [14]:
dict_merged_gpsc = dict()

for merged_gpscs in merged_gpscs_with_merge_clusters:
    base_gpsc = min(merged_gpscs, key=int)
    merge_history = ';'.join(sorted(merged_gpscs, key=int))
    for gpsc in merged_gpscs:
        dict_merged_gpsc.update({gpsc: {'merge_history': merge_history, 'base_gpsc': base_gpsc}})

In [15]:
def update_base_gpsc_and_merge_history(row):
    if row["GPSC"] == "235;9":
        row["merge_history"] = "235;9"
        return row
    
    base_gpsc = min(row["GPSC"].split(";"), key=int)

    if base_gpsc not in dict_merged_gpsc:
        row["merge_history"] = base_gpsc
        return row

    row["GPSC"] = dict_merged_gpsc[base_gpsc]["base_gpsc"]
    row["merge_history"] = dict_merged_gpsc[base_gpsc]["merge_history"]
    return row

In [16]:
df_new_db_gpsc = df_new_db_gpsc[["sample", "GPSC"]].apply(update_base_gpsc_and_merge_history, axis=1)

In [17]:
new_db_gpsc_output_path = os.path.join(dbs_path, f"{new_db}_external_clusters.csv")

df_new_db_gpsc.to_csv(new_db_gpsc_output_path, index=False)